# Multiple Instance Learning

In [1]:
#import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import metrics
from datetime import datetime
from sklearn.linear_model import LogisticRegressionCV
import seaborn as sn
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LogisticRegression
sns.set()

In [2]:
#read the dataser
musk = pd.read_csv('Musk1.csv', header=None)

Task:
Suggest two alternative bag-level representations for the given multiple instance learning problem.
Based on the proposed bag-level representations for Musk1 dataset, evaluate at least two reasonable
classifiers (of your choice). The example representation is very simple and introduced for illustration
purposes. Please do not consider it as a valid alternative. You are expected to justify your bag-level
representation approach. This may be achieved by some descriptive analysis you perform (to motivate
your approaches). You can benefit from the literature as long as you provide your references
accordingly. Please note that if the proposed representation approach has certain parameters, these
parameters are also part of your algorithm. Hence, representation parameters (if there is any) should
also be tuned together with the parameters of your proposed approach. Specify the best set of parameter
combination for the proposed representations and your classifier. Use the accuracy based on 10-fold
cross-validation on the training data as your primary performance metric in your evaluations.


In [3]:
#rename thee dataset
musk_features=musk.iloc[:,2:]
musk_features.columns = ['F{}'.format(i) for i in range(0,len(musk_features.columns))]
musk1=musk.iloc[:,:2]
musk1.set_axis(['Bag class', 'Bag Id'], axis=1, inplace=True)
musk=pd.concat([musk1,musk_features], axis=1)
print(musk.shape)
musk.head()

(476, 168)


,Bag class,Bag Id,F0,F1,F2,F3,F4,F5,F6,F7,...,F156,F157,F158,F159,F160,F161,F162,F163,F164,F165
0,1,1,42,-198,-109,-75,-117,11,23,-88,...,-238,-74,-129,-120,-38,30,48,-37,6,30
1,1,1,42,-191,-142,-65,-117,55,49,-170,...,-238,-302,60,-120,-39,31,48,-37,5,30
2,1,1,42,-191,-142,-75,-117,11,49,-161,...,-238,-73,-127,-120,-38,30,48,-37,5,31
3,1,1,42,-198,-110,-65,-117,55,23,-95,...,-238,-302,60,-120,-39,30,48,-37,6,30
4,1,2,42,-198,-102,-75,-117,10,24,-87,...,-238,-73,-127,51,128,144,43,-30,14,26


For the representation, statistical values for each id can be used. I prefer to use max and min values for a representation. For another quartiles for each id are used.

# Max-Min representation

In [4]:
#find max and min values for each bag id. sum the values and divide by two to obtain binary class
max_musk = musk.groupby("Bag Id").max().reset_index()
min_musk = musk.groupby("Bag Id").min().reset_index()
min_max_data = (min_musk+max_musk)/2
print(min_max_data.shape)
min_max_data.head()


(92, 168)


,Bag Id,Bag class,F0,F1,F2,F3,F4,F5,F6,F7,...,F156,F157,F158,F159,F160,F161,F162,F163,F164,F165
0,1.0,1.0,42.0,-194.5,-125.5,-70.0,-117.0,33.0,36.0,-129.0,...,-238.0,-187.5,-34.5,-120.0,-38.5,30.5,48.0,-37.0,5.5,30.5
1,2.0,1.0,42.0,-194.5,-122.0,-70.0,-117.0,32.5,36.0,-128.5,...,-238.0,-186.0,-33.0,51.0,126.0,143.5,43.0,-30.5,14.0,27.0
2,3.0,1.0,42.0,-166.0,-102.5,-30.5,-116.5,5.0,27.5,-142.5,...,-235.5,-57.5,-45.5,13.0,120.5,133.5,55.5,-32.5,2.0,16.5
3,4.0,1.0,43.0,-147.0,-84.5,-31.0,-117.0,26.0,20.5,-130.0,...,-237.5,-174.0,-34.0,-87.0,-34.5,31.5,51.0,-38.5,4.5,32.5
4,5.0,1.0,63.0,-51.0,-102.0,-20.0,48.5,23.5,27.5,-143.5,...,-238.5,-165.0,-54.0,-7.0,64.5,113.5,78.0,-23.0,2.0,44.5


# Quarters representation

In [5]:
#find 1st and 2nd quartile for each bag id. sum the values and divide by two to obtain binary class
firstq_musk = musk.groupby("Bag Id").quantile(0.25).reset_index()
second2_musk = musk.groupby("Bag Id").quantile(0.75).reset_index()
quartes_data = (firstq_musk+second2_musk)/2
print(quartes_data.shape)
quartes_data.head()

(92, 168)


0.25,Bag Id,Bag class,F0,F1,F2,F3,F4,F5,F6,F7,...,F156,F157,F158,F159,F160,F161,F162,F163,F164,F165
0,1.0,1.0,42.00,-194.50,-125.875,-70.000,-117.00,33.00,36.000,-128.25,...,-238.000,-187.875,-33.75,-120.0,-38.500,30.125,48.000,-37.000,5.500,30.125
1,2.0,1.0,42.00,-194.50,-122.000,-70.000,-117.00,33.25,36.375,-127.75,...,-238.000,-186.375,-32.25,51.0,126.750,143.500,42.625,-30.125,14.000,26.250
2,3.0,1.0,42.00,-166.00,-102.500,-30.500,-116.50,5.00,27.500,-142.50,...,-235.500,-57.500,-45.50,13.0,120.500,133.500,55.500,-32.500,2.000,16.500
3,4.0,1.0,42.50,-158.50,-73.750,-48.000,-117.00,20.50,22.250,-133.00,...,-237.250,-126.000,-5.50,-77.0,-33.250,31.250,50.000,-38.250,5.750,32.250
4,5.0,1.0,60.75,-83.25,-102.000,-27.875,-75.25,16.75,29.750,-143.50,...,-237.375,-93.375,-63.75,11.0,81.375,148.375,63.750,-28.625,-1.375,32.875


# Linear regression with alternative 1

In [28]:
#seperate the data into test and train set
from sklearn.utils import shuffle
import random
random.seed(123)
df = shuffle(min_max_data)

df = df.drop(df.columns[0], axis=1)
train_data = df.iloc[:70,:]
test_data = df.iloc[70:,:]

X_train1 = train_data.drop(train_data.columns[0], axis=1)
Y_train1 = train_data['Bag class']

X_test1 = test_data.drop(test_data.columns[0], axis=1)
Y_test1 = test_data['Bag class']
test_data.shape

(22, 167)

In [29]:
#logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train1, np.ravel(Y_train1))

C:\Users\90505\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
#calculate the treshold
th=Y_train1.sum()/len(X_train1)
th=float(th)
th

0.5

In [31]:
#make prediction on test set
probs_y=model.predict_proba(X_test1) 
probs_y=(probs_y[:,1]>th)*1

In [32]:
#2-by-2 confusion matrix
conf_matrix = metrics.confusion_matrix(Y_test1, probs_y)
print("Our confusion matrix is \n", pd.DataFrame(conf_matrix))

Our confusion matrix is 
    0  1
0  8  2
1  3  9


In [33]:
accuracy1=(conf_matrix[0][0]+conf_matrix[1][1])/conf_matrix.sum()
print("Accuracy is= %.3f" % accuracy1)

Accuracy is= 0.773


In [34]:
#train logistic regression model with lasso penalties
model_lasso = LogisticRegressionCV(cv=10, penalty="l1", max_iter=1000, scoring='neg_mean_absolute_error', solver='liblinear')
model_lasso.fit(X_train1, np.ravel(Y_train1))
pred_y_lasso = model_lasso.predict_proba(X_test1)

In [35]:
#calculate the loss due to the chosen lambda values by the 10-fold cross validation above
pred_y_lasso = (pred_y_lasso[:,1]>th)*1
score = metrics.log_loss(Y_test1, pred_y_lasso)
print("best score=", score) 

best score= 4.709869490098285


In [36]:
#2-by-2 confusion matrix for LR lasso model
conf_matrix_lasso = metrics.confusion_matrix(Y_test1, pred_y_lasso)
print("Our confusion matrix is \n", pd.DataFrame(conf_matrix_lasso))

Our confusion matrix is 
    0   1
0  9   1
1  2  10


In [37]:
#accuracy calculation
accuracy2=(conf_matrix_lasso[0][0]+conf_matrix_lasso[1][1])/conf_matrix_lasso.sum()
print("Accuracy is= %.3f" % accuracy2)

Accuracy is= 0.864


# Logistic regression with alternative 2

In [38]:
random.seed(3)
df = shuffle(quartes_data)

df = df.drop(df.columns[0], axis=1)
train_data = df.iloc[:70,:]
test_data = df.iloc[70:,:]

X_train2 = train_data.drop(train_data.columns[0], axis=1)
Y_train2 = train_data['Bag class']

X_test2 = test_data.drop(test_data.columns[0], axis=1)
Y_test2 = test_data['Bag class']
test_data.shape

(22, 167)

In [39]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train2, np.ravel(Y_train2))

C:\Users\90505\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
#calculate the treshold
th=Y_train2.sum()/len(X_train2)
th=float(th)
th

0.4714285714285714

In [41]:
#make prediction on test set
probs_y=model.predict_proba(X_test2) 
probs_y=(probs_y[:,1]>th)*1

In [42]:
#2-by-2 confusion matrix
conf_matrix = metrics.confusion_matrix(Y_test2, probs_y)
print("Our confusion matrix is \n", pd.DataFrame(conf_matrix))

Our confusion matrix is 
    0   1
0  6   2
1  2  12


In [43]:
accuracy1=(conf_matrix[0][0]+conf_matrix[1][1])/conf_matrix.sum()
print("Accuracy is= %.3f" % accuracy1)

Accuracy is= 0.818


In [44]:
#train logistic regression model with lasso penalties
model_lasso = LogisticRegressionCV(cv=10, penalty="l1", max_iter=1000, scoring='neg_mean_absolute_error', solver='liblinear')
model_lasso.fit(X_train2, np.ravel(Y_train2))
pred_y_lasso = model_lasso.predict_proba(X_test2)

In [45]:
#calculate the loss due to the chosen lambda values by the 10-fold cross validation above
pred_y_lasso = (pred_y_lasso[:,1]>th)*1
score = metrics.log_loss(Y_test2, pred_y_lasso)
print("best score=", score) 

best score= 6.2798865623606055


In [46]:
#2-by-2 confusion matrix for LR lasso model
conf_matrix_lasso = metrics.confusion_matrix(Y_test2, pred_y_lasso)
print("Our confusion matrix is \n", pd.DataFrame(conf_matrix_lasso))

Our confusion matrix is 
    0   1
0  5   3
1  1  13


In [64]:
#accuracy calculation
accuracy2=(conf_matrix_lasso[0][0]+conf_matrix_lasso[1][1])/conf_matrix_lasso.sum()
print("Accuracy is= %.3f" % accuracy2)

Accuracy is= 0.818


# Decision tree with alternative 1

In [49]:
#find out if there any NA in the dataset
musk.isnull().values.any()

False

In [50]:
#decide the optimum depth value by 10 fold cross validation
from sklearn import tree
depth = []
for i in range(3,20):
    clf = tree.DecisionTreeClassifier(max_depth=i)
    # Perform 10-fold cross validation 
    scores = cross_val_score(estimator=clf, X=X_train1, y=Y_train1, cv=10, n_jobs=4)
    depth.append((i,scores.mean()))
depth=pd.DataFrame(depth)
print(depth)

     0         1
0    3  0.733333
1    4  0.708333
2    5  0.679167
3    6  0.670833
4    7  0.725000
5    8  0.637500
6    9  0.720833
7   10  0.675000
8   11  0.700000
9   12  0.700000
10  13  0.745833
11  14  0.712500
12  15  0.729167
13  16  0.729167
14  17  0.700000
15  18  0.670833
16  19  0.687500


In [57]:
from sklearn.tree import DecisionTreeClassifier
clf_model = DecisionTreeClassifier(criterion="gini", random_state=6, max_depth=4, min_samples_leaf=5)   
clf_model.fit(X_train1, Y_train1)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=6, splitter='best')

In [58]:
y_predict = clf_model.predict(X_test1)

In [59]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Accuracy is= %.3f" % accuracy_score(Y_test1,y_predict))

Accuracy is= 0.727


# Decision tree with alternative 2

In [60]:
depth = []
for i in range(3,20):
    clf = tree.DecisionTreeClassifier(max_depth=i)
    # Perform 10-fold cross validation 
    scores = cross_val_score(estimator=clf, X=X_train2, y=Y_train2, cv=10, n_jobs=4)
    depth.append((i,scores.mean()))
depth=pd.DataFrame(depth)
print(depth)

     0         1
0    3  0.710119
1    4  0.673214
2    5  0.705952
3    6  0.720833
4    7  0.704167
5    8  0.704167
6    9  0.733333
7   10  0.720833
8   11  0.730952
9   12  0.730952
10  13  0.689881
11  14  0.720833
12  15  0.716667
13  16  0.689286
14  17  0.689881
15  18  0.729167
16  19  0.704167


In [61]:
from sklearn.tree import DecisionTreeClassifier
clf_model = DecisionTreeClassifier(random_state=42,max_depth=6, min_samples_leaf=3)   
clf_model.fit(X_train2, Y_train2)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [62]:
y_predict = clf_model.predict(X_test2)

In [63]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Accuracy is= %.3f" % accuracy_score(Y_test2,y_predict))

Accuracy is= 0.773


# Loss values
Logistic Regression

Alternative 1: 0.773

Alternative 2: 0.818

Logistic Regression Lasso

Alternative 1: 0.864

Alternative 2: 0.817

Decision Tree

Alternative 1: 0.727

Alternative 2: 0.773

The best combination is the max-min representation with Logistic Regression via lasso penalization